In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map  

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
roi = ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

In [4]:
def ndvi_mndwi(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    return image.addBands(mndwi).addBands(ndvi_mndwi)

# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

# 可视化参数设置
vis_params = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

In [5]:
# 筛选数据（使用日期，云量，研究区），去云
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2019-07-01', '2019-07-31') \
    .map(ndvi_mndwi)


second = ee.Image(landsat_images.toList(landsat_images.size()).get(1))
Map.addLayer(second, vis_params, 'second image')

In [6]:
band = 'ndvi_mndwi'

def histogram(image):
    band = 'ndvi_mndwi'
    dict = band + "_" + 'histogram'
    hist = image.select(band).reduceRegion(**{
        'reducer': ee.Reducer.histogram(100).combine(ee.Reducer.mean(),'',True).combine(ee.Reducer.variance(),'',True),
        'geometry': image.geometry(), 
        'scale': 30,
        'bestEffort': True
    })
    return hist.get(dict)

In [7]:
def sumSquare(i,histogram,counts,means,size,mean):
    aCounts = counts.slice(0, 0, i)
    aCountTotal = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCountTotal)
       
    bCounts = counts.slice(0, i, size)
    bCountTotal = bCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    bMeans = means.slice(0, i, size)
    bMean = bMeans.multiply(bCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(bCountTotal)

    return aCountTotal.multiply(aMean.subtract(mean).pow(2)).add(bCountTotal.multiply(bMean.subtract(mean).pow(2)))
# Return the DN that maximizes interclass variance in B5 (in the region).
def otsu(histogram):
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    mean = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(counts.reduce(ee.Reducer.sum(), [0]).get([0]))
    indices = ee.List.sequence(1,size)
    bss = indices.map(lambda i : sumSquare(i,histogram,counts,means,size,mean))
    # Return the mean value corresponding to the maximum BSS.
    return means.sort(bss).get([-1])

In [8]:
hist = histogram(second)
second = second.set({'threshold': otsu(hist)})
print(second.get('threshold').getInfo())

classD = second.select(band).lt(second.get('threshold').getInfo()).selfMask()
Map.addLayer(classD, {'palette': ['blue']}, 'class d')

0.5157633386606081


In [ ]:
hist = histogram(second)
second = second.set({'threshold': otsu(hist)})
print(second.get('threshold').getInfo())

In [ ]:
# def segmentImage(image):
#     band = 'ndvi_mndwi'
#     imageCol = ee.ImageCollection([])
#     for i in range(3):
#         for j in range(3):
#             coordinates = ee.Array(image.geometry().bounds().coordinates().get(0))
#             leftlon = coordinates.get([0,0]).add(ee.Number(0.82*i))
#             leftlat = coordinates.get([0,1]).add(ee.Number(0.72*j))
#             rightlon = coordinates.get([0,0]).add(ee.Number(0.82)).add(ee.Number(0.82*i))
#             rightlat = coordinates.get([0,1]).add(ee.Number(0.72)).add(ee.Number(0.72*j))
#             extent = ee.Geometry.Rectangle([leftlon,leftlat,rightlon,rightlat])
#             img = maskL8sr(image.clip(extent).clip(roi))
#             hist = histogram(img)
#             img_water = img.select(band).lt(otsu(hist))
#             imageCol = imageCol.merge(ee.ImageCollection(img_water))
#     return imageCol.mosaic()

In [ ]:
band = 'ndvi_mndwi'
image = second
imageCol = ee.ImageCollection([])
for i in range(3):
    for j in range(3):
        coordinates = ee.Array(second.geometry().bounds().coordinates().get(0))
        leftlon = coordinates.get([0,0]).add(ee.Number(0.82*i))
        leftlat = coordinates.get([0,1]).add(ee.Number(0.72*j))
        rightlon = coordinates.get([0,0]).add(ee.Number(0.82)).add(ee.Number(0.82*i))
        rightlat = coordinates.get([0,1]).add(ee.Number(0.72)).add(ee.Number(0.72*j))
        extent = ee.Geometry.Rectangle([leftlon,leftlat,rightlon,rightlat])
        img = maskL8sr(image.clip(extent).clip(roi))
        hist = histogram(img)
        print(otsu(hist))

In [ ]:
ig = segmentImage(second)
print(ig.size().getInfo())
# for i in range(ig.size().getInfo()):
#     second = ee.Image(ig.toList(ig.size()).get(i))
#     Map.addLayer(second, vis_params, 'second {}'.format(i))

In [ ]:
ig = segmentImage(second)
Map.addLayer(ig,{'palette': ['white','blue'],'min':0,'max':1},'ig')

In [ ]:
def segmentImage(image):
    band = 'ndvi_mndwi'
    imageCol = ee.ImageCollection([])
    for i in range(3):
        for j in range(3):
            coordinates = ee.Array(image.geometry().bounds().coordinates().get(0))
            leftlon = coordinates.get([0,0]).add(ee.Number(0.82*i))
            leftlat = coordinates.get([0,1]).add(ee.Number(0.72*j))
            rightlon = coordinates.get([0,0]).add(ee.Number(0.82)).add(ee.Number(0.82*i))
            rightlat = coordinates.get([0,1]).add(ee.Number(0.72)).add(ee.Number(0.72*j))
            extent = ee.Geometry.Rectangle([leftlon,leftlat,rightlon,rightlat])
#             img = maskL8sr(image.clip(extent).clip(roi))
#             hist = histogram(img)
#             img = img.set({'threshold':otsu(hist)})
            imageCol = imageCol.merge(ee.ImageCollection(img))
    return imageCol

In [ ]:
def block(imagecollection):
    

In [ ]:
ig = segmentImage(second)
for i in range(ig.size().getInfo()):
    second = ee.Image(ig.toList(ig.size()).get(i))
    Map.addLayer(second, vis_params, 'second {}'.format(i))

In [ ]:
threshold = otsu(histogram(image))
print('threshold : ', threshold.getInfo())

classD = image.select(band).lt(threshold).selfMask()
Map.addLayer(classD, {'palette': ['blue']}, 'class d')